# Session 1 - Introduction to Amazon SageMaker

TBD

## Prepare the data

We will use the MNIST dataset during this program. It's a simple dataset that we can use to train a model quickly, and gives us access to cover the most important aspects of a Machine Learning System.

MNIST is popular and relatively small, so it's easy to find pre-packaged versions of it. We aren't going to use those. During this program, we want to simulate a practical scenario where data will be stored in the filesystem. To accomplish this, we will load a pre-packaged version of MNIST, save it to disk, and upload it to an S3 bucket.

These are the steps you need to follow to prepare the data:

1. Create the S3 bucket to upload the dataset.
2. Load the MNIST dataset from the Keras built-in collection of small datasets, convert it into images, and save them to the disk.
3. Upload the dataset to the S3 bucket.


### Create an S3 bucket

We need to create an S3 bucket where we will upload the dataset we'll use during the program.

Make sure you set `BUCKET` to the name of the bucket you want to use.

In [13]:
BUCKET = "mlschool"

!aws s3api create-bucket --bucket $BUCKET

{
    "Location": "/mlschool"
}


### Create the dataset

We want to load the Keras' built-in version of MNIST and save it to disk so we can later upload it to S3.

There are 70,000 images. Running this cell will take some time, so this is the perfect moment to walk around, grab some coffee, and check social media. Fortunately, we only need to do this once.

In [8]:
import numpy as np
import tensorflow as tf

from PIL import Image
from pathlib import Path
from tensorflow.keras.datasets import mnist


def save(dataset, split, images, labels):
    """
    This function saves the handwritten digits to disk as PNG files.
    
    Every image will be saved inside a folder corresponding to 
    its label. For example, a digit from the train set representing 
    the number 3 will be saved inside the folder `~/train/3`.
    """
    
    for index, (image, label) in enumerate(zip(images, labels)):
        im = Image.fromarray(image)

        path = dataset / split / str(label)
        path.mkdir(parents=True, exist_ok=True)
        
        im.save(path / f"{index}.png")
        

# We will save the dataset in the home directory, inside a folder
# named `dataset`.
dataset = Path.home() / "dataset" 

# We want to make sure we don't generate the images if the dataset
# already exists.
if not dataset.exists():
    # Load the MNIST dataset using the Keras library. This returns the
    # dataset in numpy arrays.
    (X_train, y_train), (X_test, y_test) = mnist.load_data()

    # Use the function we created to save the data to disk.
    save(dataset, split="train", images=X_train, labels=y_train)
    save(dataset, split="test", images=X_test, labels=y_test)

### Upload dataset to S3

Now that we exported the MNIST dataset to the filesystem, we need to upload them to the S3 bucket we created. The easiest way to do this is to use the AWS CLI.

This command will also take a while to finish.

In [ ]:
!aws s3 cp $dataset s3://$BUCKET/dataset --recursive